# [Lab3] 모델 성능 평가 및 비교

이 노트북에서는 훈련된 두 모델의 성능을 평가하고 비교합니다.

## 주요 내용
- 로컬 환경에서 모델 테스트
- 모델 성능 비교 및 평가
- 혼동 행렬 및 분류 리포트 생성
- MLflow를 통한 실험 추적
- 최고 성능 모델 선택

## 1. 환경 설정 및 변수 로드

In [ ]:
# 이전 노트북에서 저장한 변수들 로드
%store -r

print("✅ 저장된 변수들을 로드했습니다.")
print(f"   - 등록된 모델 이름: {registered_model_name}")
print(f"   - 모델 버전 1: {registered_model_version_1.version}")
print(f"   - 모델 버전 2: {registered_model_version_2.version}")
print(f"   - MLflow 실행 ID: {run_id}")

In [ ]:
# 필수 라이브러리 임포트
import sagemaker
import boto3
import mlflow
import pandas as pd
import numpy as np
import xgboost as xgb
import os
from time import gmtime, strftime
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, RocCurveDisplay
from sklearn import metrics
import matplotlib.pyplot as plt
from sagemaker_studio import Project

# AWS 세션 초기화
boto_session = boto3.Session()
sess = sagemaker.Session()

print("✅ 라이브러리 임포트 및 세션 초기화 완료")

## 2. 프로젝트 및 MLflow 설정

Project 클래스를 사용하여 일관된 리소스 접근을 설정합니다.

In [ ]:
# 프로젝트 및 MLflow 설정
# 프로젝트 및 MLflow 설정
from sagemaker_studio import Project
from time import gmtime, strftime
import mlflow

# 프로젝트 설정 (MLflow 제외)
project = Project()
role = project.iam_role
domain_id = project.domain_id
project_id = project.id


print(f"   - MLflow 서버: {mlflow_name}")
mlflow.set_tracking_uri(mlflow_arn)

# 실행 이름 생성
run_suffix = strftime('%d-%H-%M-%S', gmtime())
run_name = f"data-preprocessing-{run_suffix}"

# MLflow 실행 시작
run_id = mlflow.start_run(
    run_name=run_name, 
    description="SageMaker Processing을 사용한 데이터 전처리"
).info.run_id

print(f"✅ MLflow 실행 시작: {run_name}")
print(f"   - 실행 ID: {run_id}")

## 3. 테스트 데이터 로드

전처리된 테스트 데이터를 로드하여 모델 성능을 평가합니다.

In [ ]:
# 임시 디렉토리 생성
!mkdir -p tmp

print("✅ 임시 디렉토리 생성 완료")

In [ ]:
# S3에서 테스트 데이터 다운로드
print("📥 테스트 데이터 다운로드 중...")

!aws s3 cp $test_path/test_x.csv tmp/test_x.csv
!aws s3 cp $test_path/test_y.csv tmp/test_y.csv

print("✅ 테스트 데이터 다운로드 완료")
print(f"   - 특성 데이터: tmp/test_x.csv")
print(f"   - 레이블 데이터: tmp/test_y.csv")

In [ ]:
# 테스트 데이터 로드
print("📊 테스트 데이터 로드 중...")

test_x = pd.read_csv('tmp/test_x.csv', names=[f'{i}' for i in range(59)])
test_y = pd.read_csv('tmp/test_y.csv', names=['y'])

print(f"✅ 테스트 데이터 로드 완료")
print(f"   - 테스트 샘플 수: {len(test_x)}")
print(f"   - 특성 수: {test_x.shape[1]}")
print(f"   - 양성 클래스 비율: {test_y['y'].mean():.3f}")

# 데이터 미리보기
print(f"\n📋 데이터 미리보기:")
print(f"   - 특성 데이터 형태: {test_x.shape}")
print(f"   - 레이블 분포: {test_y['y'].value_counts().to_dict()}")

## 4. 모델 로드 함수 정의

MLflow Model Registry와 S3에서 모델을 로드하는 함수들을 정의합니다.

In [ ]:
# 등록된 모델 상태 확인
try:
    client = mlflow.MlflowClient()
    
    # 모델 버전 정보 확인
    model_version_1 = client.get_model_version(
        name=registered_model_name,
        version=registered_model_version_1.version
    )
    
    print(f"📋 모델 1 정보:")
    print(f"   - 이름: {model_version_1.name}")
    print(f"   - 버전: {model_version_1.version}")
    print(f"   - 상태: {model_version_1.status}")
    print(f"   - 단계: {model_version_1.current_stage}")
    print(f"   - 소스: {model_version_1.source}")
    
    MODEL_REGISTRY_AVAILABLE = True
    
except Exception as e:
    print(f"⚠️ Model Registry 접근 실패: {e}")
    print("S3에서 직접 모델을 로드합니다.")
    MODEL_REGISTRY_AVAILABLE = False

In [ ]:
# 4-test-and-deploy.ipynb에서 사용할 모델 객체 생성
# 이 변수들은 2-training.ipynb에서 생성된 것들을 참조합니다

print("🔧 배포용 모델 객체 생성 중...")

try:
    # 2-training.ipynb에서 저장된 변수들이 있는지 확인
    if 'xgb1' in locals() and 'xgb2' in locals():
        print("✅ 훈련 단계에서 생성된 모델 객체를 사용합니다.")
        print(f"   - xgb1 모델 데이터: {xgb1.model_data}")
        print(f"   - xgb2 모델 데이터: {xgb2.model_data}")
    else:
        print("⚠️ 훈련 단계의 모델 객체가 없습니다.")
        print("   배포 단계에서 직접 모델 URI를 사용하거나")
        print("   2-training.ipynb를 다시 실행해주세요.")
        
        # 대안: 저장된 모델 URI 사용 (있다면)
        if 'model1_uri' in locals() and 'model2_uri' in locals():
            print("\n💡 저장된 모델 URI를 사용합니다:")
            print(f"   - 모델 1 URI: {model1_uri}")
            print(f"   - 모델 2 URI: {model2_uri}")
            
            # 간단한 모델 객체 생성 (URI만 포함)
            class SimpleModelRef:
                def __init__(self, model_data):
                    self.model_data = model_data
            
            xgb1 = SimpleModelRef(model1_uri)
            xgb2 = SimpleModelRef(model2_uri)
            
            print("✅ 간단한 모델 참조 객체를 생성했습니다.")
        else:
            print("\n❌ 모델 URI도 없습니다. 2-training.ipynb를 먼저 실행해주세요.")
            
except Exception as e:
    print(f"❌ 모델 객체 생성 실패: {e}")

# 변수 저장 (다음 노트북에서 사용)
if 'xgb1' in locals() and 'xgb2' in locals():
    %store xgb1
    %store xgb2
    print("\n💾 모델 객체를 저장했습니다 (xgb1, xgb2)")
else:
    print("\n⚠️ 저장할 모델 객체가 없습니다.")

In [ ]:
def load_model_from_s3(model_data_uri, model_name="model"):
    """S3에서 직접 XGBoost 모델을 로드하는 함수"""
    import boto3
    import tarfile
    import pickle as pkl
    import os
    
    try:
        print(f"📥 {model_name} S3에서 로드 중: {model_data_uri}")
        
        # S3에서 모델 다운로드
        model_file = f"./tmp/{model_name}-model.tar.gz"
        os.makedirs('./tmp', exist_ok=True)
        
        bucket, key = model_data_uri.replace("s3://", "").split("/", 1)
        boto3.client("s3").download_file(bucket, key, model_file)
        
        # 압축 해제
        with tarfile.open(model_file, "r:gz") as t:
            t.extractall(path="./tmp")
        
        # 모델 로드
        model_path = "./tmp/xgboost-model"
        with open(model_path, "rb") as f:
            model = pkl.load(f)
        
        print(f"✅ {model_name} 로드 성공")
        return model
        
    except Exception as e:
        print(f"❌ {model_name} 로드 실패: {e}")
        return None

def load_model_from_mlflow(model_name, version):
    """MLflow Model Registry에서 모델을 로드하는 함수"""
    try:
        model_uri = f"models:/{model_name}/{version}"
        print(f"📥 MLflow에서 모델 로드 중: {model_uri}")
        
        model = mlflow.xgboost.load_model(model_uri)
        print(f"✅ MLflow 모델 로드 성공")
        return model
        
    except Exception as e:
        print(f"❌ MLflow 모델 로드 실패: {e}")
        return None

print("✅ 모델 로드 함수 정의 완료")

## 5. 모델 1 테스트 (보수적 하이퍼파라미터)

첫 번째 모델을 로드하고 테스트 데이터로 예측을 수행합니다.

In [ ]:
# 첫 번째 모델 로드 및 테스트
print("🔍 첫 번째 모델 (보수적 하이퍼파라미터) 테스트")

model1 = None

# 방법 1: MLflow Model Registry에서 로드 시도
if MODEL_REGISTRY_AVAILABLE:
    model1 = load_model_from_mlflow(registered_model_name, registered_model_version_1.version)

# 방법 2: MLflow 실패 시 S3에서 직접 로드
if model1 is None:
    print("MLflow 로드 실패, S3에서 직접 로드 시도...")
    model1 = load_model_from_s3(xgb1.model_data, "model1")

# 모델 로드 성공 시 예측 수행
if model1 is not None:
    print("\n📊 첫 번째 모델로 예측 수행 중...")
    
    # XGBoost DMatrix 생성
    dtest = xgb.DMatrix(test_x)
    
    # 예측 수행
    predictions1 = model1.predict(dtest)
    predictions1 = np.array(predictions1, dtype=float).squeeze()
    
    print(f"✅ 예측 완료")
    print(f"   - 예측 샘플 수: {len(predictions1)}")
    print(f"   - 예측값 범위: {predictions1.min():.4f} ~ {predictions1.max():.4f}")
    print(f"   - 평균 예측값: {predictions1.mean():.4f}")
    
    # 예측 결과 미리보기
    print(f"\n📋 첫 10개 예측값:")
    for i in range(min(10, len(predictions1))):
        print(f"   샘플 {i+1}: {predictions1[i]:.4f}")
        
else:
    print("❌ 첫 번째 모델 로드 실패")
    predictions1 = None

## 6. 모델 2 테스트 (적극적 하이퍼파라미터)

두 번째 모델을 로드하고 테스트 데이터로 예측을 수행합니다.

In [ ]:
# 두 번째 모델 로드 및 테스트
print("🔍 두 번째 모델 (적극적 하이퍼파라미터) 테스트")

model2 = None

# 방법 1: MLflow Model Registry에서 로드 시도
if MODEL_REGISTRY_AVAILABLE:
    model2 = load_model_from_mlflow(registered_model_name, registered_model_version_2.version)

# 방법 2: MLflow 실패 시 S3에서 직접 로드
if model2 is None:
    print("MLflow 로드 실패, S3에서 직접 로드 시도...")
    model2 = load_model_from_s3(xgb2.model_data, "model2")

# 모델 로드 성공 시 예측 수행
if model2 is not None:
    print("\n📊 두 번째 모델로 예측 수행 중...")
    
    # XGBoost DMatrix 생성
    dtest = xgb.DMatrix(test_x)
    
    # 예측 수행
    predictions2 = model2.predict(dtest)
    predictions2 = np.array(predictions2, dtype=float).squeeze()
    
    print(f"✅ 예측 완료")
    print(f"   - 예측 샘플 수: {len(predictions2)}")
    print(f"   - 예측값 범위: {predictions2.min():.4f} ~ {predictions2.max():.4f}")
    print(f"   - 평균 예측값: {predictions2.mean():.4f}")
    
    # 예측 결과 미리보기
    print(f"\n📋 첫 10개 예측값:")
    for i in range(min(10, len(predictions2))):
        print(f"   샘플 {i+1}: {predictions2[i]:.4f}")
        
else:
    print("❌ 두 번째 모델 로드 실패")
    predictions2 = None

## 7. 모델 성능 비교

두 모델의 예측 결과를 비교하고 성능을 평가합니다.

In [ ]:
# 두 모델 모두 로드 성공한 경우 성능 비교
if predictions1 is not None and predictions2 is not None:
    print("📊 모델 성능 비교")
    print("=" * 50)
    
    # 이진 분류를 위한 임계값 설정 (0.5)
    pred1_binary = (predictions1 > 0.5).astype(int)
    pred2_binary = (predictions2 > 0.5).astype(int)
    
    # 실제 레이블
    y_true = test_y['y'].values
    
    # AUC 점수
    auc1 = roc_auc_score(y_true, predictions1)
    auc2 = roc_auc_score(y_true, predictions2)
    
    # 정확도
    acc1 = accuracy_score(y_true, pred1_binary)
    acc2 = accuracy_score(y_true, pred2_binary)
    
    print(f"🥇 모델 1 (보수적):")
    print(f"   - AUC: {auc1:.4f}")
    print(f"   - 정확도: {acc1:.4f}")
    
    print(f"\n🥈 모델 2 (적극적):")
    print(f"   - AUC: {auc2:.4f}")
    print(f"   - 정확도: {acc2:.4f}")
    
    # 최고 성능 모델 선택
    if auc1 > auc2:
        print(f"\n🏆 최고 성능: 모델 1 (AUC: {auc1:.4f})")
        best_model = model1
        best_predictions = predictions1
        best_model_name = "보수적 하이퍼파라미터 모델"
        best_model_version = registered_model_version_1
    else:
        print(f"\n🏆 최고 성능: 모델 2 (AUC: {auc2:.4f})")
        best_model = model2
        best_predictions = predictions2
        best_model_name = "적극적 하이퍼파라미터 모델"
        best_model_version = registered_model_version_2
        
else:
    print("⚠️ 모델 로드 실패로 인해 성능 비교를 수행할 수 없습니다.")
    # 로드된 모델이 있다면 그것을 사용
    if predictions1 is not None:
        best_predictions = predictions1
        best_model_name = "보수적 하이퍼파라미터 모델"
        best_model_version = registered_model_version_1
        print(f"모델 1만 사용합니다: {best_model_name}")
    elif predictions2 is not None:
        best_predictions = predictions2
        best_model_name = "적극적 하이퍼파라미터 모델"
        best_model_version = registered_model_version_2
        print(f"모델 2만 사용합니다: {best_model_name}")
    else:
        print("❌ 사용 가능한 모델이 없습니다.")

## 8. 혼동 행렬 및 분류 리포트

최고 성능 모델의 상세한 성능 분석을 수행합니다.

In [ ]:
# 혼동 행렬 (최고 성능 모델 기준)
if 'best_predictions' in locals():
    print(f"📋 {best_model_name} 혼동 행렬:")
    
    # 이진 분류 예측
    best_pred_binary = (best_predictions > 0.5).astype(int)
    
    # 혼동 행렬 생성
    confusion_matrix_df = pd.crosstab(
        index=test_y['y'].values, 
        columns=best_pred_binary, 
        rownames=['실제값'], 
        colnames=['예측값']
    )
    
    print(confusion_matrix_df)
    
    # 분류 리포트
    print(f"\n📊 {best_model_name} 분류 리포트:")
    print(classification_report(test_y['y'].values, best_pred_binary))
else:
    print("⚠️ 평가할 모델이 없습니다.")

## 9. 시각화 및 MLflow 실험 추적

모델 성능을 시각화하고 MLflow에 기록합니다.

In [ ]:
def plot_confusion_matrix(
    cm, class_names, title="Confusion matrix", cmap=plt.cm.Blues, normalize=False
):
    """혼동 행렬을 시각화하는 함수"""
    if normalize:
        cm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation="nearest", cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    ax.set(
        xticks=np.arange(cm.shape[1]),
        yticks=np.arange(cm.shape[0]),
        ylim=(cm.shape[0] - 0.5, -0.5),
        xticklabels=class_names,
        yticklabels=class_names,
        title=title,
        ylabel="Ground truth label",
        xlabel="Predicted label",
    )

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=30, ha="right", rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = ".2f" if normalize else "d"
    thresh = cm.max() / 2.0
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(
                j,
                i,
                format(cm[i, j], fmt),
                ha="center",
                va="center",
                color="white" if cm[i, j] > thresh else "black",
            )
    fig.tight_layout()
    return ax, fig

print("✅ 시각화 함수 정의 완료")

In [ ]:
# 혼동 행렬 시각화
if 'best_predictions' in locals():
    print(f"📊 {best_model_name} Confusion Matrix Visualization")
    
    class_names = ["no", "yes"]
    cm = confusion_matrix(test_y['y'].values, (best_predictions > 0.5).astype(int))
    
    ax, fig = plot_confusion_matrix(
        cm, 
        class_names, 
        title=f"{best_model_name} - Confusion Matrix"
    )
    
    plt.show()
    
    print("✅ 혼동 행렬 시각화 완료")
else:
    print("⚠️ 시각화할 모델이 없습니다.")
    fig = None

In [ ]:
# MLflow에 혼동 행렬 로그
if 'best_model_version' in locals() and fig is not None:
    try:
        print(f"📝 MLflow에 혼동 행렬 로그 중...")
        print(f"   - 모델: {best_model_version.name} 버전 {best_model_version.version}")
        
        mlflow.set_experiment(experiment_name)
        with mlflow.start_run(run_id=best_model_version.run_id):
            mlflow.log_figure(fig, "confusion_matrix.png")
            
            # 추가 메트릭 로그
            if predictions1 is not None and predictions2 is not None:
                mlflow.log_metric("test_auc_model1", auc1)
                mlflow.log_metric("test_auc_model2", auc2)
                mlflow.log_metric("test_accuracy_model1", acc1)
                mlflow.log_metric("test_accuracy_model2", acc2)
        
        print("✅ MLflow 로그 완료")
        
    except Exception as e:
        print(f"⚠️ MLflow 로그 실패: {e}")
else:
    print("⚠️ MLflow 로그를 위한 데이터가 없습니다.")

## 10. 결과 요약 및 저장

모델 평가 결과를 요약하고 다음 노트북에서 사용할 변수들을 저장합니다.

In [ ]:
# 결과 요약
print("📋 모델 평가 결과 요약")
print("=" * 50)

if 'best_model_name' in locals():
    print(f"🏆 최고 성능 모델: {best_model_name}")
    
    if predictions1 is not None and predictions2 is not None:
        print(f"\n📊 성능 비교:")
        print(f"   모델 1 (보수적): AUC={auc1:.4f}, 정확도={acc1:.4f}")
        print(f"   모델 2 (적극적): AUC={auc2:.4f}, 정확도={acc2:.4f}")
        
        # 성능 차이 계산
        auc_diff = abs(auc1 - auc2)
        acc_diff = abs(acc1 - acc2)
        
        print(f"\n📈 성능 차이:")
        print(f"   AUC 차이: {auc_diff:.4f}")
        print(f"   정확도 차이: {acc_diff:.4f}")
        
    print(f"\n✅ 테스트 데이터 크기: {len(test_x)} 샘플")
    print(f"✅ 양성 클래스 비율: {test_y['y'].mean():.3f}")
    
else:
    print("❌ 모델 평가를 완료하지 못했습니다.")

print("\n" + "=" * 50)
print("🎯 모델 평가 완료!")

In [ ]:
# 다음 노트북에서 사용할 변수들 저장
if 'best_model_name' in locals():
    %store best_model_name
    %store best_model_version
    
    if 'best_predictions' in locals():
        %store best_predictions
    
    print("✅ 변수 저장 완료:")
    print(f"   - best_model_name: {best_model_name}")
    print(f"   - best_model_version: {best_model_version.version}")
    print(f"   - best_predictions: {len(best_predictions) if 'best_predictions' in locals() else 'N/A'} 개")
else:
    print("⚠️ 저장할 변수가 없습니다.")